<a href="https://colab.research.google.com/github/bu11ymaguire/studying/blob/main/ReinforcementLearning/GridWorld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GridWorld

In [17]:
import numpy as np

class GridWorld:
  def __init__(self):
    self.action_space = [0,1,2,3]
    self.action_meaning = {
        0: "UP",
        1: "DOWN",
        2: "LEFT",
        3: "RIGHT"
    }

    self.reward_map = np.array(
        [[0,0,0,1.0],
         [0,None,0,-1.0],
            [0,0,0,0]
        ]
    )

    self.goal_stat = (0,3) #목표 지점
    self.wall_state = (1,1) #벽
    self.start_state = (2,0) #시작 지점
    self.agent_state = self.start_state

  @property
  def height(self):
    return len(self.reward_map)

  @property
  def width(self):
    return len(self.reward_map[0])

  @property
  def shape(self):
    return self.reward_map.shape

  def actions(self):
    return self.action_space

  def states(self):
    for h in range(self.height):
      for w in range(self.width):
        yield(h,w)

  def next_state(self, state, action):
    action_move_map = [(-1,0),(1,0),(0,1),(0,-1)]
    move = action_move_map[action]
    next_state = (state[0]+ move[0],state[1] + move[1])
    ny, nx = next_state

    if nx < 0 or nx >= self.width or ny < 0 or ny >= self.height: #이동하게 될 위치가 영역의 밖인 경우?
      next_state = state
    elif next_state == self.wall_state: #이동하게 될 위치가 벽인 경우
      next_state = state

    return next_state

  def reward(self, state, action, next_state):
    return self.reward_map[next_state]


In [16]:
env = GridWorld()

for _ in env.actions():
  print(_)

print('===')

for state in env.states():
  print(state)

0
1
2
3
===
(0, 0)
(0, 1)
(0, 2)
(0, 3)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
